In [1]:
# !pip install pycocotools
# !pip install ffmpeg
#!pip install ipywidgets

In [2]:
from IPython.display import Video, display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras import layers, models
from tqdm.auto import tqdm

In [3]:
CLASS_NAME = ['Paper', 'Rock', 'Scissor']

In [4]:
def video_graber():
    video_grab = Video(r"C:\Users\Mewandi\Desktop\MI\Neural_Network\video.mp4", width=400, height=300)
    return video_grab

In [5]:
def capture_img(num_images=50, label='Null'):
    img = []
    y = []

    for x in tqdm(range(num_images)):
        new_img = np.array(video_graber(0))
        img.append(new_img)

        if label != 'Null':
           y.append(label)

    img = np.array(img)
    y = np.array(y)
    return img, y

In [6]:
paper_img, paper_label = capture_img(num_images=50, label='0')

  0%|          | 0/50 [00:00<?, ?it/s]

TypeError: video_graber() takes 0 positional arguments but 1 was given

In [ ]:
rock_img, rock_label = capture_img(num_images=50, label='1')

In [ ]:
scissor_img, scissor_label = capture_img(num_images=50, label='2')

In [ ]:
train_img = np.concatenate((paper_img, rock_img, scissor_img))
train_img = train_img / 255
train_label = np.concatenate((paper_label, rock_label, scissor_label))

### Data Generator to add more images

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagenerator = ImageDataGenerator(
                                    rotation_range = 40,
                                    width_shift_range = 0.2,
                                    height_shift_range = 0.2,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True,
                                    file_mode = 'nearest', # To fill the empty spaces in video
                                  )

#### Make new trains in 3D shapes

In [ ]:
idx = 0
new_train_imgs = []
new_train_labels = []

for image in range(train_img):
    img = image.reshape((1,)+image.shape) # make 3D shape. (1, ) it supposed to be like (1,400,300) but the 2d size is given in video function

    i = 0
    for batch in datagenerator.flow(img, save_prefix = 'test', save_format = 'jpeg'):
        plt.figure(i)
        plot = plt.imshow(batch[0])
        i += 1
        new_train_imgs.append(batch[0])
        new_train_labels.append(train_label[idx]) # for adding the index into img
    
        if i > 10: # Each image should be added 10 differents shapes
            break
    idx += 1

plt.show

In [ ]:
new_train_imgs = np.array(new_train_imgs)
new_train_labels = np.array(new_train_labels)

## Now we create the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D

In [ ]:
model = Sequential()
 
model.add(Conv2D(32, (5,5), activation='relu', input_shape=(400,300,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (5,5), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (5,5), activation='relu'))

In [ ]:
model.summary()

In [ ]:
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3)) # Outputs are (paper, rock, scissor) 

In [ ]:
model.summary()

#### Training the model

In [ ]:
model.compile(optimizer='adam', loss = tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

history = model.fit(new_train_imgs, new_train_labels, epochs=10)

In [ ]:
test_image, _ = capture_img(1) #  _ is because we did not add the label parameter into capture_img()
test_image = test_image / 255

prediction = model.predict(test_image)
plt.imshow(test_image[0])
plt.title(CLASS_NAME[np.argmax(prediction[0])]) # is 